In [ ]:
%matplotlib inline

from shared_content import *

In [ ]:
# Do you want to write figures to png files? Careful, it will overwrite!
write_figures = True
figdir = 'Figures/'

In [ ]:
filedir = 'anafiles/'

anatype = 'MCBNBRecoTrackExiting'
#anatype = 'MCBNBMCTrackExiting'
#!ls -ltra $filedir | tail -n 10

In [ ]:
df_dict, segdf_dict = {}, {}
reses = [ 0, 1, 2, 3, 5 ]
for seglen in reses:
    myfile = 'MCSBiasStudy_%s_anaout_14cmseg_%dres_bothscatters_nonrelfix_realdedx1_highlandconstantMOMENTUMDEPENDENT.root' %( anatype, seglen )
    df_dict[seglen], segdf_dict[seglen] = get_dfs(filedir + myfile)

In [ ]:
def basic_comparison_fig(xvar, yvar, plotname =None, extraquery = None, addtext = None, seglen = 10, myres = 2):
    plt.figure(figsize=(10,6)) 
    mydf = df_dict[myres]
    myquery = 'true_E < 99999999'
    if extraquery is not None: myquery += ' and %s'%extraquery
    myx = mydf.query(myquery)[xvar].values
    myy = mydf.query(myquery)[yvar].values
    blah = plt.hist2d(myx,myy,bins=((np.linspace(0,4,50),np.linspace(0,4,50))),cmin=1)
    blah = plt.colorbar()
    blah = plt.grid(True)
    blha = plt.xlabel('%s'%titles[xvar],fontsize=16)
    blha = plt.ylabel('%s'%titles[yvar],fontsize=16)
    blha = plt.title('%s'%(titles[anatype]),fontsize=16)
    blha.set_y(1.04)
    blah = plt.plot([0,100],[0,100],'g--',linewidth=3)
    
    if addtext is not None:
        plt.text(plt.xlim()[1]*0.6, plt.ylim()[1]*0.1, addtext, fontsize=20)
     
    plt.text(plt.xlim()[1]*0.5, plt.ylim()[1]*0.1, '%d mrad Resolution' % myres,fontsize=20)
    
    if write_figures:
        if plotname is not None: 
            print "Saving figure %s"%plotname
            plt.tight_layout()
            plt.savefig(figdir + plotname)
        else: print "YOU WANTED TO SAVE A PLOT BUT DIDN'T GIVE A PLOT NAME!"


In [ ]:
for res in reses:
    
    basic_comparison_fig(xvar='true_momentum',yvar='full_MCS_momentum',
                         plotname=None, seglen=10, myres = res)

In [ ]:
#reco-true/true
def fractional_bias_plot(xvar = 'full_range_momentum', xbins = np.linspace(0.35,1,10),
                   yvar = 'full_MCS_momentum', slicevar = 'full_range_momentum',
                         plot_bin_distributions = False, extraquery = None, slicetitlebase = None,
                        slicebins = np.linspace(-0.8,.8,20),
                        biasplotname = None, resplotname = None, biasmainfig_ylims = None,
                         resmainfig_ylims = None,
                        usegausfit = False):
    plt.figure(figsize=(10,6))
    binning = xbins
    binwidth = float(binning[1]-binning[0])
    bincenters = binning + (binwidth/2)

    for res in reses:
        
        myreses, mystds, myerrs_bias, myerrs_res = [], [], [], []
    
        for x in xrange(len(binning)-1):
            
            binmin = binning[x]
            binmax = binning[x+1]
            #print "binmin = %0.2f, binmax = %0.2f"% ( binmin, binmax )
            myquery = '%s > %f and %s < %f'%(slicevar,binmin,slicevar,binmax)
            if anatype == 'DataBNBSelectedRecoTrack': 
                myquery += ' and not maybe_bad'
            if extraquery is not None:
                myquery += ' and %s' % extraquery
                
            mydf = df_dict[res].query(myquery)
            true = mydf[xvar].values
            reco = mydf[yvar].values
            mymean = ((reco-true)/true).mean()
            mystd = ((reco-true)/true).std()
            
            if plot_bin_distributions:
                #plt.figure(figsize=(5,3))
                datahist = np.histogram((reco-true)/true,bins=slicebins)
                #titlestring = '$\\frac{%s - %s}{%s}$ for $%s$ in %0.2f $\\rightarrow$ %0.2f GeV'% \
                #(latextitles[yvar],latextitles[xvar],latextitles[xvar],latextitles[slicevar],binmin,binmax)
                #t =plt.title(titlestring,fontsize=16)
                #move the title up a bit
                #t.set_y(1.04) 
                #plt.grid(False)
                
                # Plot gaussian on each bin distribution
                if usegausfit:
                     
                    slicebinvals = datahist[0]
                    slicebincenters = [ datahist[1][x] + (datahist[1][x+1]-datahist[1][x])/2 for x in xrange(len(datahist[1])-1) ]
        
                    # Fit a normal distribution
                    gmod = Model(gaussian)
                    #initial random guesses of 1, 1, 2
                    result = gmod.fit(slicebinvals, x=slicebincenters, amp=1, cen=1, wid=2)
    
                    thisx = np.linspace(np.min(slicebins),np.max(slicebins),100)
                    thisy = gaussian(thisx, result.params['amp'], result.params['cen'], result.params['wid'])
                    #plt.plot(thisx, thisy, 'g-',\
                    #     label='Gaus Fit',\
                    #     linewidth=2)
                    #addtext = 'Fit: \n$\sigma$ = %0.2f, \n$\mu$ = %0.2f'%\
                    #(np.abs(result.params['wid']),result.params['cen'])
                    #plt.text(plt.xlim()[1]*0.35, plt.ylim()[1]*0.1, addtext, fontsize=14)
                    
                    # If use gaus fit, use the result of that instead of straight mean and RMS
                    mymean = result.params['cen']
                    #Somehow when there are like 2 data points you get a negative width?!
                    mystd  = np.abs(result.params['wid'])
                    # If the fit doesn't converge, 'wid' will be the initial guess of 2.0... don't use these points then
                    # (this only happens when no entries in the sliced histogram)
                    if int(result.params['wid']) == 2:
                        print "WARNING: FIT DIDN'T CONVERGE!"
                        mymean = ((reco-true)/true).mean()
                        mystd = ((reco-true)/true).std()
                        
                 
                    
                    
                #plt.xlabel('$\\frac{%s - %s}{%s}$'%(latextitles[yvar],latextitles[xvar],latextitles[xvar]),fontsize=16)
                #plt.ylabel('Counts',fontsize=16)
                #plt.xlim((np.min(slicebins),np.max(slicebins)))
                #plt.legend(loc=1)
                #if write_figures and slicetitlebase is not None:
                #    fullfigname = figdir + slicetitlebase + '_slice_%0.2f_%0.2f.png'%(binmin,binmax)
                #    print '\n\n WRITING A FIGURE!! %s\n\n'%fullfigname
                #    plt.tight_layout()
                #    plt.savefig(fullfigname)
            
            
            myerr_bias = mystd / np.sqrt( float(len(true)) )
            myerr_res = mystd / np.sqrt( float(2*len(true)) )
            myreses.append( mymean )
            mystds.append( mystd )
            myerrs_bias.append( myerr_bias )
            myerrs_res.append( myerr_res )
            
            
        #BIAS PLOT    
        plt.errorbar(bincenters[:-1],myreses,yerr=myerrs_bias,xerr=binwidth/2,fmt='o',label='Resolution: %d mrad'%res)
    plt.ylabel('GausMean($\\frac{%s - %s}{%s}$)'%(latextitles[yvar],latextitles[xvar],latextitles[xvar]),fontsize=25)
    plt.xlabel('%s'%titles[slicevar],fontsize=15)
    plt.grid(True)
    plt.legend(loc='best')
    t = plt.title('Fractional Bias: %s'%titles[anatype],fontsize=16)
    t.set_y(1.04)
    if biasmainfig_ylims is not None:
        blah = plt.ylim(biasmainfig_ylims)
    if write_figures and biasplotname is not None:
        print " \n\n Writing the main bias figure!! %s\n\n" % (figdir+biasplotname)
        plt.tight_layout()
        plt.savefig(figdir + biasplotname)

In [ ]:
#reco-true/true
def fractional_resolution_plot(xvar = 'full_range_momentum', xbins = np.linspace(0.35,1,10),
                   yvar = 'full_MCS_momentum', slicevar = 'full_range_momentum',
                         plot_bin_distributions = False, extraquery = None, slicetitlebase = None,
                        slicebins = np.linspace(-0.8,.8,20),
                        biasplotname = None, resplotname = None, biasmainfig_ylims = None,
                         resmainfig_ylims = None,
                        usegausfit = False):
    plt.figure(figsize=(10,6))
    binning = xbins
    binwidth = float(binning[1]-binning[0])
    bincenters = binning + (binwidth/2)

    for res in reses:
        
        myreses, mystds, myerrs_bias, myerrs_res = [], [], [], []
    
        for x in xrange(len(binning)-1):
            
            binmin = binning[x]
            binmax = binning[x+1]
            #print "binmin = %0.2f, binmax = %0.2f"% ( binmin, binmax )
            myquery = '%s > %f and %s < %f'%(slicevar,binmin,slicevar,binmax)
            if anatype == 'DataBNBSelectedRecoTrack': 
                myquery += ' and not maybe_bad'
            if extraquery is not None:
                myquery += ' and %s' % extraquery
                
            mydf = df_dict[res].query(myquery)
            true = mydf[xvar].values
            reco = mydf[yvar].values
            mymean = ((reco-true)/true).mean()
            mystd = ((reco-true)/true).std()
            
            if plot_bin_distributions:
                #plt.figure(figsize=(5,3))
                datahist = np.histogram((reco-true)/true,bins=slicebins)
                #titlestring = '$\\frac{%s - %s}{%s}$ for $%s$ in %0.2f $\\rightarrow$ %0.2f GeV'% \
                #(latextitles[yvar],latextitles[xvar],latextitles[xvar],latextitles[slicevar],binmin,binmax)
                #t =plt.title(titlestring,fontsize=16)
                #move the title up a bit
                #t.set_y(1.04) 
                #plt.grid(False)
                
                # Plot gaussian on each bin distribution
                if usegausfit:
                     
                    slicebinvals = datahist[0]
                    slicebincenters = [ datahist[1][x] + (datahist[1][x+1]-datahist[1][x])/2 for x in xrange(len(datahist[1])-1) ]
        
                    # Fit a normal distribution
                    gmod = Model(gaussian)
                    #initial random guesses of 1, 1, 2
                    result = gmod.fit(slicebinvals, x=slicebincenters, amp=1, cen=1, wid=2)
    
                    thisx = np.linspace(np.min(slicebins),np.max(slicebins),100)
                    thisy = gaussian(thisx, result.params['amp'], result.params['cen'], result.params['wid'])
                    #plt.plot(thisx, thisy, 'g-',\
                    #     label='Gaus Fit',\
                    #     linewidth=2)
                    #addtext = 'Fit: \n$\sigma$ = %0.2f, \n$\mu$ = %0.2f'%\
                    #(np.abs(result.params['wid']),result.params['cen'])
                    #plt.text(plt.xlim()[1]*0.35, plt.ylim()[1]*0.1, addtext, fontsize=14)
                    
                    # If use gaus fit, use the result of that instead of straight mean and RMS
                    mymean = result.params['cen']
                    #Somehow when there are like 2 data points you get a negative width?!
                    mystd  = np.abs(result.params['wid'])
                    # If the fit doesn't converge, 'wid' will be the initial guess of 2.0... don't use these points then
                    # (this only happens when no entries in the sliced histogram)
                    if int(result.params['wid']) == 2:
                        print "WARNING: FIT DIDN'T CONVERGE!"
                        mymean = ((reco-true)/true).mean()
                        mystd = ((reco-true)/true).std()
                        
                 
                    
                    
                #plt.xlabel('$\\frac{%s - %s}{%s}$'%(latextitles[yvar],latextitles[xvar],latextitles[xvar]),fontsize=16)
                #plt.ylabel('Counts',fontsize=16)
                #plt.xlim((np.min(slicebins),np.max(slicebins)))
                #plt.legend(loc=1)
                #if write_figures and slicetitlebase is not None:
                #    fullfigname = figdir + slicetitlebase + '_slice_%0.2f_%0.2f.png'%(binmin,binmax)
                #    print '\n\n WRITING A FIGURE!! %s\n\n'%fullfigname
                #    plt.tight_layout()
                #    plt.savefig(fullfigname)
            
            
            myerr_bias = mystd / np.sqrt( float(len(true)) )
            myerr_res = mystd / np.sqrt( float(2*len(true)) )
            myreses.append( mymean )
            mystds.append( mystd )
            myerrs_bias.append( myerr_bias )
            myerrs_res.append( myerr_res )
            
            
        #RESOLUTION PLOT    
        plt.errorbar(bincenters[:-1],mystds,yerr=myerrs_bias,xerr=binwidth/2,fmt='o',label='Resolution: %d mrad' % res)
    plt.ylabel('GausWidth($\\frac{%s - %s}{%s}$)'%(latextitles[yvar],latextitles[xvar],latextitles[xvar]),fontsize=25)
    plt.xlabel('%s'%titles[slicevar],fontsize=15)
    plt.grid(True)
    plt.legend(loc='best')
    t = plt.title('Fractional Momentum Resolution: %s'%titles[anatype],fontsize=16)
    t.set_y(1.04)
    if resmainfig_ylims is not None:
        blah = plt.ylim(resmainfig_ylims)
    if write_figures and resplotname is not None:
        print " \n\n Writing the main res figure!! %s\n\n" % (figdir+resplotname)
        plt.tight_layout()
        plt.savefig(figdir + resplotname)

In [ ]:
df_dict[0].columns.values

In [ ]:
if 'Exiting' not in anatype:
    fractional_bias_plot(xvar='full_range_momentum_inverse',yvar='full_MCS_momentum_inverse',
                         xbins=np.linspace(0.35,2,10),
                           plot_bin_distributions = True,
                           slicevar = 'full_range_momentum',
                           slicetitlebase = None,
                           slicebins = np.linspace(-0.8,0.8,50),
                           biasplotname = 'resstudy_%s_bias.png'%anatype,
                           biasmainfig_ylims = (-.10,.10),
                            usegausfit = True)
    
    fractional_resolution_plot(xvar='full_range_momentum_inverse',yvar='full_MCS_momentum_inverse',
                         xbins=np.linspace(0.35,2,10),
                           plot_bin_distributions = True,
                           slicevar = 'full_range_momentum',
                           slicetitlebase = None,
                           slicebins = np.linspace(-0.8,0.8,50),
                           resplotname = 'resstudy_%s_resolution.png'%anatype,
                           resmainfig_ylims = (0,.15),
                            usegausfit = True)

In [ ]:
if 'Exiting' in anatype:
    
    fractional_bias_plot(xvar='true_momentum_inverse',yvar='full_MCS_momentum_inverse',
                         xbins=np.linspace(0.35,4,10),
                           plot_bin_distributions = True,
                           slicevar = 'true_momentum',
                           slicetitlebase = None,
                           slicebins = np.linspace(-0.8,0.8,50),
                           biasplotname = 'resstudy_%s_bias.png'%anatype,
#                            biasmainfig_ylims = (-.5,.10),
                           biasmainfig_ylims = (-.2,.20),
                            usegausfit = True)
    
    fractional_resolution_plot(xvar='true_momentum_inverse',yvar='full_MCS_momentum_inverse',
                         xbins=np.linspace(0.35,4,10),
                           plot_bin_distributions = True,
                           slicevar = 'true_momentum',
                           slicetitlebase = None,
                           slicebins = np.linspace(-0.8,0.8,50),
                           resplotname = 'resstudy_%s_resolution.png'%anatype,
                           resmainfig_ylims = (0,0.30),
                            usegausfit = True)

In [ ]:
def deflection_plot(binmin,binmax,nbins,extraquery=None,plotname=None, addtext=None, figname = None, 
                    customtitle = None, highland_constant = 12.265, cut2percent = False, use_true_p = False, detres = 0):
    
    fig = plt.figure(figsize=(10,6))
    
    poop = plt.grid(True)
    
    myquery = 'run < 99999999'
    if extraquery is not None: myquery += ' and %s'%extraquery
    print myquery
    mybins = np.linspace(binmin,binmax,nbins)
    
    segdf_dict[detres]['tempycol'] = segdf_dict[detres]['delta_theta_y']/np.sqrt(np.square(segdf_dict[detres]['predicted_RMS_fromMCS'])-np.square(detres))
    segdf_dict[detres]['tempxcol'] = segdf_dict[detres]['delta_theta_x']/np.sqrt(np.square(segdf_dict[detres]['predicted_RMS_fromMCS'])-np.square(detres))

    thisquery1 = '%s > %f and %s < %f'%('tempycol',binmin,'tempycol',binmax)
    thisquery2 = '%s > %f and %s < %f'%('tempxcol',binmin,'tempxcol',binmax)

    myvals = np.append(
        segdf_dict[detres].query(myquery).\
        query(thisquery1)['tempycol'].values,
        segdf_dict[detres].query(myquery).\
        query(thisquery2)['tempxcol'].values
    )
    
    #testing out changing the 13.6 in the highland formula...
    # it's /= because the highland formula is in the denominator
    myvals /= highland_constant/13.6
    
    mystd = np.std(myvals)
    mymean = np.mean(myvals)

    datahist = plt.hist(myvals,bins=mybins,normed=True,
                             alpha=0.5,label='$\Delta\\theta/RMS$ Values')#,
                            #weights = myweights)
   
    datahist_nonorm = np.histogram(myvals,bins=mybins,normed=False)
    
    binvals = datahist[0]
    bincenters = [ datahist[1][x] + (datahist[1][x+1]-datahist[1][x])/2 for x in xrange(len(datahist[1])-1) ]
    
    tempbinvals = binvals
    tempbincenters = bincenters
    if cut2percent is True:
        #work your way out from both sides removing bins until you're left with 98% of the central area
        totalentries = float(np.sum(binvals))
        subset = 0.02 * totalentries
        outercount = 0
        while outercount < subset:
            outercount += tempbinvals[0]
            outercount += tempbinvals[-1]
            tempbinvals = np.delete(tempbinvals,0)
            tempbinvals = np.delete(tempbinvals,-1)
            tempbincenters = np.delete(tempbincenters,0)
            tempbincenters = np.delete(tempbincenters,-1)
        
        binvals = tempbinvals
        bincenters = tempbincenters
        
    
    # Fit a normal distribution
    gmod = Model(gaussian)
    #initial random guesses of 1, 1, 2
    result = gmod.fit(binvals, x=bincenters, amp=1, cen=1, wid=2)

    #print(result.fit_report())
    #print help(result)
    #plt.plot(bincenters, binvals,         'bo')
    #plt.plot(bincenters, result.init_fit, 'k--')
    print result.params
    plt.plot(bincenters, result.best_fit, 'g-',\
             label='Gaussian Fit: $\sigma$ = %0.4f' % result.params['wid'],\
            linewidth=4)
  
    #data_mu, data_std = norm.fit(myvals)
    
    #myhighland = highlandE(data_std)

    # Plot the PDF.
    #x = np.linspace(binmin,binmax,100)
    #p = norm.pdf(x, data_mu, data_std)
    #plt.plot(x, p, 'g', linewidth=4,label='Gaussian Fit to Data')
    
    
    plt.title('%s'%titles[anatype],fontsize=14)
   
    if customtitle is not None:
        plt.title(customtitle,fontsize=14)
        
    plt.xlabel('Delta Theta / RMS',fontsize=16)
    blah = plt.legend(loc='best')
    leg = plt.legend(loc=2)
    
    if addtext is not None:
        plt.text(plt.xlim()[1]*0.25, plt.ylim()[1]*0.85, addtext, fontsize=20)
        
    if write_figures:
        fullfigname = figdir + 'Highland_validation_%s.png' % anatype
        if figname is not None:
            fullfigname = figdir + figname
            
        print "\n\n WRITING FIGURE %s! \n\n"%fullfigname
        plt.tight_layout()
        if plotname is not None: plt.savefig(figdir + plotname)
        else: plt.savefig(fullfigname)
            
    return result.params['wid']

In [ ]:
# determining the detector resolution constant for reconstructed tracks
if anatype == 'MCBNBRecoTrack':
    myreses = []
    mywidths = []
    myextraquery = None
    for myres in sorted(segdf_dict.keys()):
        mywidths.append(deflection_plot(-5,5,50,extraquery=myextraquery,cut2percent=False,use_true_p=False,detres=myres))
        myreses.append(myres)
        
    plt.figure(figsize=(10,6))
    plt.plot(myreses,mywidths,'ro--')
    plt.title('Calibrating the Detector Resolution Term for Reco Tracks',fontsize=16)
    plt.xlabel('(Negative) Detector Resolution Term',fontsize=16)
    plt.ylabel('Gaussian Fit Width',fontsize=16)
    plt.grid(True)